In [2]:
import datetime
import time
import os
import sys
import pytz
from pathlib import Path

import numpy as np
import pandas as pd

import astroplan
import skyfield

import astropy.units as u

from astropy.coordinates import EarthLocation
from astropy.time import Time

from skyfield import api
from skyfield import almanac

import matplotlib
from matplotlib import style
style.use('ggplot')
import matplotlib.pyplot as plt

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

ts = api.load.timescale()
e = api.load('de421.bsp')
tz = pytz.timezone("America/Phoenix")

In [4]:
def nearest_minute(dt):
    # USNO predictions round to nearest minute so we use this hack to follow that
    return (dt + datetime.timedelta(seconds=30)).replace(second=0, microsecond=0)

In [6]:
lat = "31:41:19.6"
lon = "-110:53:04.4"
elevation = 2600 * u.m
location = EarthLocation.from_geodetic(lon, lat, elevation)
mmt = astroplan.Observer(name="MMTO", location=location, timezone="US/Arizona", pressure=0*u.mbar)
tset = mmt.sun_set_time(Time(datetime.datetime.now()), which='next', horizon=-0.8333*u.deg)
tset.isot

'2019-12-19T00:22:39.086'

In [7]:
mmt_sf = api.Topos(latitude_degrees=location.lat.value, longitude_degrees=location.lon.value, elevation_m=2600.0)

In [17]:
year = 2019

# get new moons bracketing requested year
t0 = ts.utc(year-1, 11, 1)
t1 = ts.utc(year+1, 3, 1)
phase_times, phase_flags = almanac.find_discrete(t0, t1, almanac.moon_phases(e))
newmoons = phase_times[phase_flags == 0]

date_range = pd.date_range(start=f"11/1/{year}", end=f"12/11/{year}", tz=tz)
usno_horizon = -(5/6) * u.deg
horizons = {
    "6 Deg": -6 * u.deg,
    "12 Deg": -12 * u.deg,
    "18 Deg": -18 * u.deg
}

header1 = f"     {year}          Sun     Sun     Sun    RA 3H                RA 3H    Sun     Sun     Sun               Moon    Moon   Moon @ Midnight"
header2 = " Date    Sunset   6 Deg   12 Deg  18 Deg  West at  Sid Time    East at 18 Deg  12 Deg  6 Deg  Sunrise     rise    set    Illum     Age  "
header3 = "                  W Hrz   W Hrz   W Hrz   18 Deg   Midnight    18 Deg  E Hrz   E Hrz   E Hrz                               %       Days\n"

print(header1)
print(header2)
print(header3)
for d in date_range:
     # almanac day is MST, but its night falls into the next day in UT
    night_start = Time(f"{str(d.date())} 00:00:00") + 1 * u.day
    
    d_str = d.date().strftime("%b %d")
    
    tset = mmt.sun_set_time(night_start, which='next', horizon=usno_horizon)
    tset_str = nearest_minute(tset.to_datetime(timezone=tz)).strftime("%-H %M")
    tset_twilights = {}
    for k, h in horizons.items():
        tset_twilights[k] = mmt.sun_set_time(night_start, which='next', horizon=h)
        
    eve_str = ""
    for t in tset_twilights:
        eve_str += nearest_minute(tset_twilights[t].to_datetime(timezone=tz)).strftime("%-H %M")
        eve_str += "   "

    ra3_west = tset_twilights['18 Deg'].sidereal_time(kind='apparent', longitude=mmt.location.lon) - 3 * u.hourangle + (30/3600) * u.hourangle  # add 30 sec to round to nearest minute
    ra3_west = ra3_west.wrap_at(24 * u.hourangle)
    ra3_west_str = "{:02d} {:02d}".format(int(ra3_west.hms.h), int(ra3_west.hms.m))
    
    # set up local midnight for the date we're calculating
    midnight = Time(f"{str(d.date())} 07:00:00") + 1 * u.day
    midnight_st = (Time(f"{str(d.date())} 07:00:00") + 1 * u.day).sidereal_time(kind='apparent', longitude=mmt.location.lon).to_string(sep=' ', precision=0)

    trise = mmt.sun_rise_time(night_start, which='next', horizon=usno_horizon)
    trise_str = nearest_minute(trise.to_datetime(timezone=tz)).strftime("%-H %M").format("{:5s}")
    trise_twilights = {}
    for k, h in horizons.items():
        trise_twilights[k] = mmt.sun_rise_time(night_start, which='next', horizon=h)
         
    ra3_east = trise_twilights['18 Deg'].sidereal_time(kind='apparent', longitude=mmt.location.lon) + 3 * u.hourangle + (30/3600) * u.hourangle  # add 30 sec to round to nearest minute
    ra3_east = ra3_east.wrap_at(24 * u.hourangle)
    ra3_east_str = "{:02d} {:02d}".format(int(ra3_east.hms.h), int(ra3_east.hms.m))

    morn_str = ""
    for t in list(trise_twilights.keys())[::-1]:
        morn_str += nearest_minute(trise_twilights[t].to_datetime(timezone=tz)).strftime("%-H %M")
        morn_str += "    "
    
    moon_rise = mmt.moon_rise_time(night_start, which='next', horizon=usno_horizon)
    moon_set = mmt.moon_set_time(night_start, which='next', horizon=usno_horizon)

    if moon_rise < trise and moon_rise > tset:
        mr_str = nearest_minute(moon_rise.to_datetime(timezone=tz)).strftime("%-H %M")
    else:
        mr_str = "     "
    if moon_set > tset and moon_set < trise:
        ms_str = nearest_minute(moon_set.to_datetime(timezone=tz)).strftime("%-H %M")
    else:
        ms_str = "     "
    
    moon_ill = "{:3d}".format(int(round(100 * mmt.moon_illumination(midnight))))
    newmoon_diff = midnight.to_datetime(timezone=pytz.utc) - newmoons.utc_datetime()
    nearest = abs(newmoon_diff).argmin()
    age = newmoon_diff[nearest].total_seconds()/86400
    age_str = "{:5.1f}".format(age)
    #print(f"{d_str}    {tset_str}   {eve_str}{ra3_west_str}     {midnight_st}    {ra3_east_str}   {morn_str}{trise_str}     {mr_str}   {ms_str}   {moon_ill}    {age_str}")
    outstr = "{:6s}    {:5s}   {:24s}{:5s}     {:8s}    {:5s}   {:24s}{:5s}     {:5s}   {:5s}   {:3s}     {:5s}".format(
        d_str,
        tset_str,
        eve_str,
        ra3_west_str,
        midnight_st,
        ra3_east_str,
        morn_str,
        trise_str,
        mr_str,
        ms_str,
        moon_ill,
        age_str
    )
    print(outstr)

     2019          Sun     Sun     Sun    RA 3H                RA 3H    Sun     Sun     Sun               Moon    Moon   Moon @ Midnight
 Date    Sunset   6 Deg   12 Deg  18 Deg  West at  Sid Time    East at 18 Deg  12 Deg  6 Deg  Sunrise     rise    set    Illum     Age  
                  W Hrz   W Hrz   W Hrz   18 Deg   Midnight    18 Deg  E Hrz   E Hrz   E Hrz                               %       Days

Nov 01    17 34   17 59   18 28   18 57   18 17     2 21 31     10 40   5 18    5 46    6 15    6 40              21 38    30       5.1
Nov 02    17 33   17 59   18 27   18 56   18 21     2 25 28     10 45   5 19    5 47    6 16    6 41              22 34    39       6.1
Nov 03    17 33   17 58   18 27   18 55   18 24     2 29 24     10 50   5 19    5 48    6 17    6 42              23 29    49       7.1
Nov 04    17 32   17 57   18 26   18 54   18 27     2 33 21     10 54   5 20    5 49    6 18    6 43              0 25     58       8.1
Nov 05    17 31   17 56   18 25   18 54   18 

In [42]:
tset.to_datetime(timezone=tz)

datetime.datetime(2019, 1, 9, 17, 37, 15, 591415, tzinfo=<DstTzInfo 'America/Phoenix' MST-1 day, 17:00:00 STD>)

In [47]:
str(d.date())

'2019-01-10'

In [82]:
t_test = Time(f"{str(d.date())} 07:00:00")
t3 = t_test.sidereal_time(kind='apparent', longitude=mmt.location.lon) + 3 * u.hourangle + (30/3600) * u.hourang

<Angle 9.91702755 hourangle>

In [83]:
f"{int(t3.hms.h)} {int(t3.hms.m)}"

'9 55'

In [86]:
t1 = d.date()

In [90]:
t1.strftime("%b %d")

'Jan 10'

In [112]:
a = tset_twilights['18 Deg'].sidereal_time(kind='apparent', longitude=mmt.location.lon) - 3 * u.hourangle + (30/3600) * u.hourangle
a.wrap_at(24 * u.hourangle)

<Angle 23.0392965 hourangle>

In [116]:
a.to_string(sep=' ', precision=0)

'-0 57 39'

In [129]:
l = list(trise_twilights.keys())
l.reverse()
l

['18 Deg', '12 Deg', '6 Deg']

In [203]:
diff = midnight.to_datetime(timezone=pytz.utc) - newmoons.utc_datetime()
idx = abs(diff).argmin()
age = diff[idx]
age

datetime.timedelta(days=5, seconds=19908, microseconds=779000)

In [205]:
age.total_seconds()/86400

5.23042568287037

In [207]:
24*3600

86400

In [222]:
delta = -age.total_seconds()/86400

In [223]:
"{:5.1f}".format(delta)

' -5.2'

In [11]:
newmoons

<Time tt=[2458430.1688849684 ... 2458903.1480309167] len=17>

In [14]:
[t.utc_iso() for t in newmoons]

['2018-11-07T16:02:02Z',
 '2018-12-07T07:20:21Z',
 '2019-01-06T01:28:11Z',
 '2019-02-04T21:03:35Z',
 '2019-03-06T16:03:58Z',
 '2019-04-05T08:50:29Z',
 '2019-05-04T22:45:30Z',
 '2019-06-03T10:01:57Z',
 '2019-07-02T19:16:13Z',
 '2019-08-01T03:11:55Z',
 '2019-08-30T10:37:09Z',
 '2019-09-28T18:26:22Z',
 '2019-10-28T03:38:28Z',
 '2019-11-26T15:05:35Z',
 '2019-12-26T05:13:08Z',
 '2020-01-24T21:42:00Z',
 '2020-02-23T15:32:01Z']

In [15]:
t1 = Time("2019-11-26T15:05:35Z")
t2 = Time("2019-12-11T07:00:00Z")

In [16]:
t1 - t2

<TimeDelta object: scale='tai' format='jd' value=-14.662789351851853>